In [1]:
!pip install pyngrok

In [2]:
!ngrok authtoken '2iJkFXO5IgR67Wml0o38Qg2OXY2_4qKY1hmTNhJ9JF53Ptf4i'  # Replace with your token from ngrok.com

Traceback (most recent call last):
  File "/usr/local/bin/ngrok", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py", line 617, in main
    run(sys.argv[1:])
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py", line 603, in run
    install_ngrok(pyngrok_config)
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py", line 99, in install_ngrok
    installer.install_ngrok(pyngrok_config.ngrok_path, ngrok_version=pyngrok_config.ngrok_version)
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/installer.py", line 123, in install_ngrok
    download_path = _download_file(url, **kwargs)
                    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/installer.py", line 274, in _download_file
    response = urlopen(url, **kwargs)
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/urllib/request.py", line 216, in urlopen
    return ope

In [ ]:
!ngrok tcp 8765

Traceback (most recent call last):
  File "/usr/lib/python3.11/subprocess.py", line 1264, in wait
    return self._wait(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 2053, in _wait
    (pid, sts) = self._try_wait(0)
                 ^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/subprocess.py", line 2011, in _try_wait
    (pid, sts) = os.waitpid(self.pid, wait_flags)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/ngrok", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py", line 617, in main
    run(sys.argv[1:])
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/ngrok.py", line 605, in run
    process.run_process(pyngrok_config.ngrok_path, args)
  File "/usr/local/lib/python3.11/dist-packages/pyngrok/pro

In [7]:
# Install dependencies in Colab
!pip install ultralytics websockets opencv-python torch supervision

  Using cached ultralytics-8.3.105-py3-none-any.whl.metadata (37 kB)
  Using cached supervision-0.25.1-py3-none-any.whl.metadata (14 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cache

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Change to your desired directory
%cd /content/drive/MyDrive/Stream_YOLO

/content/drive/MyDrive/Stream_YOLO


In [5]:
def get_center_of_bbox(bbox):
    x1,y1,x2,y2 = bbox
    return int((x1+x2)/2),int((y1+y2)/2)

def get_bbox_width(bbox):
    return bbox[2]-bbox[0]

def measure_distance(p1,p2):
    return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2)**0.5

def measure_xy_distance(p1,p2):
    return p1[0]-p2[0],p1[1]-p2[1]

def get_foot_position(bbox):
    x1,y1,x2,y2 = bbox
    return int((x1+x2)/2),int(y2)

In [6]:
from sklearn.cluster import KMeans

class TeamAssigner:
    def __init__(self):
        self.team_colors = {}
        self.player_team_dict = {}

    def get_clustering_model(self,image):
        # Reshape the image to 2D array
        image_2d = image.reshape(-1,3)

        # Preform K-means with 2 clusters
        kmeans = KMeans(n_clusters=2, init="k-means++",n_init=1)
        kmeans.fit(image_2d)

        return kmeans

    def get_player_color(self,frame,bbox):
        image = frame[int(bbox[1]):int(bbox[3]),int(bbox[0]):int(bbox[2])]

        top_half_image = image[0:int(image.shape[0]/2),:]

        # Get Clustering model
        kmeans = self.get_clustering_model(top_half_image)

        # Get the cluster labels forr each pixel
        labels = kmeans.labels_

        # Reshape the labels to the image shape
        clustered_image = labels.reshape(top_half_image.shape[0],top_half_image.shape[1])

        # Get the player cluster
        corner_clusters = [clustered_image[0,0],clustered_image[0,-1],clustered_image[-1,0],clustered_image[-1,-1]]
        non_player_cluster = max(set(corner_clusters),key=corner_clusters.count)
        player_cluster = 1 - non_player_cluster

        player_color = kmeans.cluster_centers_[player_cluster]

        return player_color


    def assign_team_color(self,frame, player_detections):

        player_colors = []
        for _, player_detection in player_detections.items():
            bbox = player_detection["bbox"]
            player_color =  self.get_player_color(frame,bbox)
            player_colors.append(player_color)

        kmeans = KMeans(n_clusters=2, init="k-means++",n_init=10)
        kmeans.fit(player_colors)

        self.kmeans = kmeans

        self.team_colors[1] = kmeans.cluster_centers_[0]
        self.team_colors[2] = kmeans.cluster_centers_[1]


    def get_player_team(self,frame,player_bbox,player_id):
        if player_id in self.player_team_dict:
            return self.player_team_dict[player_id]

        player_color = self.get_player_color(frame,player_bbox)

        team_id = self.kmeans.predict(player_color.reshape(1,-1))[0]
        team_id+=1

        if player_id ==91:
            team_id=1

        self.player_team_dict[player_id] = team_id

        return team_id

In [8]:
from ultralytics import YOLO
import supervision as sv
import pickle
import os
import numpy as np
import pandas as pd
import cv2
import sys
sys.path.append('../')


class Tracker:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.tracker = sv.ByteTrack()

    def add_position_to_tracks(sekf,tracks):
        for object, object_tracks in tracks.items():
            for frame_num, track in enumerate(object_tracks):
                for track_id, track_info in track.items():
                    bbox = track_info['bbox']
                    if object == 'ball':
                        position= get_center_of_bbox(bbox)
                    else:
                        position = get_foot_position(bbox)
                    tracks[object][frame_num][track_id]['position'] = position

    def detect_frames(self, frames):
        batch_size=5
        detections = []
        for i in range(0,len(frames),batch_size):
            detections_batch = self.model.predict(frames[i:i+batch_size],conf=0.1)
            detections += detections_batch
        return detections

    def get_object_tracks(self, frames, read_from_stub=False, stub_path=None):

        if read_from_stub and stub_path is not None and os.path.exists(stub_path):
            with open(stub_path,'rb') as f:
                tracks = pickle.load(f)
            return tracks

        detections = self.detect_frames(frames)

        tracks={
            "players":[],
            "referees":[],
            "ball":[]
        }

        for frame_num, detection in enumerate(detections):
            cls_names = detection.names
            cls_names_inv = {v:k for k,v in cls_names.items()}

            # Covert to supervision Detection format
            detection_supervision = sv.Detections.from_ultralytics(detection)

            # Convert GoalKeeper to player object
            for object_ind , class_id in enumerate(detection_supervision.class_id):
                if cls_names[class_id] == "goalkeeper":
                    detection_supervision.class_id[object_ind] = cls_names_inv["player"]

            # Track Objects
            detection_with_tracks = self.tracker.update_with_detections(detection_supervision)

            tracks["players"].append({})
            tracks["referees"].append({})
            tracks["ball"].append({})

            for frame_detection in detection_with_tracks:
                bbox = frame_detection[0].tolist()
                cls_id = frame_detection[3]
                track_id = frame_detection[4]

                if cls_id == cls_names_inv['player']:
                    tracks["players"][frame_num][track_id] = {"bbox":bbox}

                if cls_id == cls_names_inv['referee']:
                    tracks["referees"][frame_num][track_id] = {"bbox":bbox}

            for frame_detection in detection_supervision:
                bbox = frame_detection[0].tolist()
                cls_id = frame_detection[3]

                if cls_id == cls_names_inv['ball']:
                    tracks["ball"][frame_num][1] = {"bbox":bbox}

        if stub_path is not None:
            with open(stub_path,'wb') as f:
                pickle.dump(tracks,f)

        return tracks

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
import os
import asyncio
import cv2
import numpy as np
import base64
import websockets
import json
import torch
from ultralytics import YOLO
import nest_asyncio
import time
from collections import deque

# Apply nest_asyncio to allow nested event loops in Colab
nest_asyncio.apply()

# Set LD_LIBRARY_PATH for NVIDIA libraries (if needed)
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})


# Verify GPU availability
if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
    device = torch.device("cuda")
else:
    print("GPU is not available, falling back to CPU")
    device = torch.device("cpu")

# Function to run processes asynchronously and print output
async def run_process(cmd):
    print('>>> starting', *cmd)
    p = await asyncio.subprocess.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE,
    )

    async def pipe(lines):
        async for line in lines:
            print(line.strip().decode('utf-8'))

    await asyncio.gather(
        pipe(p.stdout),
        pipe(p.stderr),
    )

team_assigner = TeamAssigner()

frame_buffer = deque(maxlen=5)
team_assigned = False


# WebSocket server to process frames and send back detections
async def process_frames(websocket, path=None):
    global team_assigned
    # Load the YOLOv8 model and move it to the GPU
    model = YOLO("best.pt")  # Your trained YOLOv8 model
    model.to(device)  # Explicitly move the model to the GPU (or CPU if GPU isn't available)
    while True:
        try:
            # Receive base64-encoded frame from client
            message = await websocket.recv()
            if not message:
                break
            # Decode the frame
            img_data = base64.b64decode(message)
            np_arr = np.frombuffer(img_data, np.uint8)
            frame = cv2.imdecode(np_arr, cv2.IMREAD_COLOR)
            frame_buffer.append(frame)

            # Run YOLO inference with GPU acceleration
            detections = []

                       # === Tracking & Team Assignment Integration ===
            if len(frame_buffer) == 5:
                # Initialize Tracker
                tracker = Tracker('best.pt')
                tracks = tracker.get_object_tracks(list(frame_buffer))
                tracker.add_position_to_tracks(tracks)

                latest_players = tracks["players"][-1]
                latest_refs = tracks["referees"][-1]
                latest_ball = tracks["ball"][-1].get(1)

                # Assign teams once when enough players appear
                if not team_assigned and len(latest_players) >= 4:
                    team_assigner.assign_team_color(frame, latest_players)
                    team_assigned = True

                enriched_detections = []

                for track_id, data in latest_players.items():
                    team = None
                    if team_assigned:
                        team = team_assigner.get_player_team(frame, data["bbox"], track_id)
                    enriched_detections.append({
                        "box": [float(x) for x in data["bbox"]],
                        "label": int(2),  # player
                        "conf": float(1.0),
                        "track_id": int(track_id),
                        "team": int(team),
                        "color": [int(c) for c in data.get("team_color", (0, 0, 255))]
                    })

                for track_id, data in latest_refs.items():
                    enriched_detections.append({
                        "box": [float(x) for x in data["bbox"]],
                        "label": int(3),  # referee
                        "conf": float(1.0),
                        "track_id": int(track_id)
                    })

                if latest_ball:
                    enriched_detections.append({
                        "box": [float(x) for x in latest_ball["bbox"]],
                        "label": int(0),  # ball
                        "conf": float(1.0)
                    })

                # Override `detections` with enriched data
                detections = enriched_detections


            # Send detection results back
            await websocket.send(json.dumps(detections))
        except websockets.exceptions.ConnectionClosed:
            print("Client disconnected")
            break

# Start the WebSocket server
async def start_websocket_server():
    server = await websockets.serve(
        process_frames,  # Directly pass the function (no lambda)
        "0.0.0.0",
        8765
    )
    print("WebSocket server started on ws://0.0.0.0:8765")
    await server.wait_closed()
# Main function to run ngrok and WebSocket server
async def main():
    # Set ngrok authtoken
    await asyncio.gather(
        run_process(['ngrok', 'config', 'add-authtoken', ''])
    )

    # Run WebSocket server and ngrok tunnel
    await asyncio.gather(
        start_websocket_server(),
        run_process(['ngrok', 'tcp', '8765', '--log', 'stderr'])
    )

# Run the main function
asyncio.run(main())

GPU is not available, falling back to CPU
>>> starting ngrok config add-authtoken 2iJkFXO5IgR67Wml0o38Qg2OXY2_4qKY1hmTNhJ9JF53Ptf4i
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
>>> starting ngrok tcp 8765 --log stderr
WebSocket server started on ws://0.0.0.0:8765
t=2025-04-09T19:15:35+0000 lvl=info msg="no configuration paths supplied"
t=2025-04-09T19:15:35+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
t=2025-04-09T19:15:35+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
t=2025-04-09T19:15:35+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040 allow_hosts=[]
t=2025-04-09T19:15:35+0000 lvl=info msg="client session established" obj=tunnels.session
t=2025-04-09T19:15:35+0000 lvl=info msg="tunnel session started" obj=tunnels.session
t=2025-04-09T19:15:35+0000 lvl=info msg="started tunnel" obj=tunnels name=command_line addr=//localhost:8765 url=tcp://4.tcp.ngrok.io:14

KeyboardInterrupt: 

In [ ]:
# Start WebSocket server
start_server = websockets.serve(process_frames, "0.0.0.0", 8765)
asyncio.get_event_loop().run_until_complete(start_server)
asyncio.get_event_loop().run_forever()